In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy import fftpack
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pickle

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [474]:
#pose estimation
train = pd.DataFrame()
test = pd.DataFrame()

test = pd.read_csv("../data/ensemble/pose/samitha_q_test.txt", header=None)
train = pd.read_csv("../data/ensemble/pose/samitha_q_train.txt", header=None)

train_df = train.sample(frac=1).reset_index(drop=True)
test_df = test.sample(frac=1).reset_index(drop=True)
print("train size: " + str(train.shape) + "    test size: " + str(test.shape))


x_test = test_df.loc[:, :14]
y_test = test_df.loc[:, 15]

x_train = train_df.loc[:, :14]
y_train = train_df.loc[:, 15]

x_combined = x_train.append(x_test)
x_combined = (x_combined - x_combined.mean()) / x_combined.std() + 0.000001
x_combined = x_combined.reset_index(drop=True)

x_train = x_combined.loc[:x_train.shape[0] - 1, :]
x_test = x_combined.loc[x_train.shape[0]:, :]

p_x_test = x_test
p_y_test = y_test

nn_score = 0.0
pose_model = None
for i in range(10):
    clf = MLPClassifier(activation='relu', hidden_layer_sizes=(200, 200), max_iter=200000, learning_rate='adaptive', early_stopping=False)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    if score > nn_score:
        nn_score = score
        pose_model = clf

print('\nNN score: ' + str(nn_score))
pkl_filename = "pose_pickle.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(pose_model, file)

train size: (37260, 16)    test size: (18630, 16)

NN score: 0.574503488996


In [475]:
#activity
test = pd.read_csv("../data/ensemble/activity/samitha_test.txt", header=None)
train = pd.read_csv("../data/ensemble/activity/samitha_train.txt", header=None)

train_df = train.sample(frac=1).reset_index(drop=True)
test_df = test.sample(frac=1).reset_index(drop=True)

print("train size: " + str(train.shape) + "    test size: " + str(test.shape))

x_test = test_df.loc[:,:24]
y_test = test_df.loc[:,25]

x_train = train_df.loc[:,:24]
y_train = train_df.loc[:,25]

x_combined = x_train.append(x_test)
x_combined =(x_combined-x_combined.mean())/x_combined.std()+0.000001
x_combined = x_combined.reset_index(drop=True)

x_train = x_combined.loc[:x_train.shape[0]-1, :]
x_test = x_combined.loc[x_train.shape[0]:, :]

a_x_test = x_test
a_y_test = y_test
    
nn_score = 0.0
activity_model = None
for i in range(10):
    clf = MLPClassifier(activation='relu', hidden_layer_sizes=(100, 100), max_iter=200000, learning_rate='adaptive',
                        early_stopping=False)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    if score > nn_score:
        nn_score = score
        activity_model = clf

print('NN score: ' + str(nn_score))  
pkl_filename = "activity_pickle.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(activity_model, file)

train size: (35280, 26)    test size: (17640, 26)
NN score: 0.656179138322


In [476]:
print("posture")
p_y_pred = pose_model.predict(p_x_test)
print(accuracy_score(p_y_test, p_y_pred))
print(confusion_matrix(p_y_test, p_y_pred))

posture
0.574503488996
[[1977    0    0    0    0    5    0    0    0   18]
 [   0   30   10 1960    0    0    0    0    0    0]
 [   0  846 1057   97    0    0    0    0    0    0]
 [   0 1878    0  122    0    0    0    0    0    0]
 [   0    0    0    0 2000    0    0    0    0    0]
 [ 416    0    0    0    0  144    0    0    0   70]
 [   0    0    0    0    0    0 1929    0    2   69]
 [   0    0  409    0  278    0    0 1313    0    0]
 [ 565   52  192   16    3    2   25    0 1081   64]
 [  30   87    7   74    0  745    0    0    7 1050]]


In [484]:
with open("activity_pickle.pkl", 'rb') as file:  
    activity_pickle = pickle.load(file)

a_y_pred = activity_pickle.predict(a_x_test)
print(accuracy_score(a_y_test, a_y_pred))
print(confusion_matrix(a_y_test, a_y_pred))

0.656179138322
[[1666    4   56   14    1   47    0  113    0    0]
 [   0 1514  338   49    0    0    0    0    0    0]
 [   0    0 1901    0    0    0    0    0    0    0]
 [  17   15  152 1642    2    5    0    3   60    5]
 [1519    9    0    0  290    1    0   77    5    0]
 [   3   13    0   49   13  234  146    0   73    0]
 [   0    0    0    0    0   14 1887    0    0    0]
 [   3    0    0    0   23    0    0 1846   29    0]
 [  92   21    0  928   38  137    0    0  586   99]
 [  31    0    0    0  257    0    0  973  631    9]]


In [327]:
print("activity")
a_y_pred = activity_model.predict(a_x_test)
print(accuracy_score(a_y_test, a_y_pred))
print(confusion_matrix(a_y_test, a_y_pred))

activity
0.638492063492
[[1686    7  110   25    0    5    0   68    0    0]
 [   0 1444  389   68    0    0    0    0    0    0]
 [   0    0 1899    0    0    2    0    0    0    0]
 [  11   33  139 1595   13    1    0    0   80   29]
 [1422    6    0    4  423    0    0   39    7    0]
 [  11   19    0   50   55  185  164    0   47    0]
 [   0    0    0    0    0   12 1889    0    0    0]
 [  33    0    7    0   36    0    0 1612  213    0]
 [ 120   12    0  962   54  105    0    0  523  125]
 [  28    0    0    0  204    0    0  848  814    7]]


In [328]:
dat1 = a_x_test.reset_index(drop=True)
dat2 = a_y_test.reset_index(drop=True)
activity_test = pd.concat([dat1, dat2], axis=1)

dat1 = p_x_test.reset_index(drop=True)
dat2 = p_y_test.reset_index(drop=True)
pose_test = pd.concat([dat1, dat2], axis=1)

print(pose_test.groupby([15]).size())
print(activity_test.groupby([25]).size())

15
bottle_drinking                     2000
hammer_hammering                    2000
knife_chopping                      2000
knife_cutting                       2000
mug_drinking                        2000
none                                 630
pen_writing                         2000
saw_sawing                          2000
screwdriver_screwing(no-release)    2000
spoon_stirring                      2000
dtype: int64
25
bottle_drinking                  1901
hammer_hammering                 1901
knife_chopping                   1901
knife_cutting                    1901
mug_drinking                     1901
none                              531
pen_writing                      1901
saw_sawing                       1901
screwdriver_screwing(release)    1901
spoon_stirring                   1901
dtype: int64


In [466]:
p_test = pose_test[pose_test[15].isin(['spoon_stirring'])]
p_test = p_test.iloc[99:, :]
p_test = p_test.reset_index(drop=True)
a_test = activity_test[activity_test[25].isin(['spoon_stirring'])]
a_test = a_test.reset_index(drop=True)

p_x_test = p_test.loc[:,:14]
p_y_test = p_test.loc[:,15]

a_x_test = a_test.loc[:,:24]
a_y_test = a_test.loc[:,25]

p_y_pred = pose_model.predict(p_x_test)
p_score = accuracy_score(p_y_test, p_y_pred)

a_y_pred = activity_model.predict(a_x_test)
a_score = accuracy_score(a_y_test, a_y_pred)

p = pose_model.predict_proba(p_x_test)
a = activity_model.predict_proba(a_x_test)

print(p_test.shape)
print(a_test.shape)

(1901, 16)
(1901, 26)


In [467]:
activity_list = ['DB', 'HA', 'CH', 'CK', 'DM', 'RE', 'WR', 'SA', 'SC', 'ST']
posture_list = ['DB', 'HA', 'CH', 'CK', 'DM', 'RE', 'WR', 'SA', 'SC', 'ST']
results = []
result_df = pd.DataFrame(columns=['activity', 'posture', 'ensemble'])
os.remove('ensemble_result.csv')

for r in range(p.shape[0]):
    
    activity_accuracy = []
    posture_accuracy = []
    prediction = ''

    for i in range(p[r].shape[0]):
        posture_accuracy.append((posture_list[i], p[r][i]))

    for i in range(a[r].shape[0]):
        activity_accuracy.append((activity_list[i], a[r][i]))

    max_activity = sorted(activity_accuracy,key=lambda x: x[1], reverse=True)[:3]
    max_posture = sorted(posture_accuracy,key=lambda x: x[1], reverse=True)[:3]
    
    if max_posture[0][0] in (['DB', 'DM', 'WR']) and max_posture[0][1]>0.95:
        prediction = max_posture[0][0]
     
    elif max_posture[0][1] > max_activity[0][1] and max_activity[0][1] - max_activity[1][1] < 0.9:
        prediction = max_posture[0][0]
    else:
        if max_activity[0][1] - max_activity[1][1] > 0.8:
            prediction = max_activity[0][0]
        elif max_posture[0][1] > 0.5:
            prediction = max_posture[0][0]
        else:
            prediction = max_activity[0][0]
            
    result_df = result_df.append({'activity': max_activity[0][0], 'posture':max_posture[0][0], 'ensemble':prediction},  ignore_index=True)

    results.append(str(max_activity) + "|" + str(max_posture))
    line = str(max_activity) + ',' + str(max_posture)
    with open('ensemble_result.csv', "a") as write_file:
        line = line.replace("[", "").replace("]", "").replace('\'', '').replace('(', '').replace(')', '')
        write_file.write(line + "\n")


ensemble
CK      7
DB     19
DM    119
HA     17
RE    118
SA    769
SC    709
ST    143
dtype: int64

In [335]:
print(result_df.groupby(['posture']).size())

posture
BO    1859
SP      42
dtype: int64


In [336]:
print(result_df.groupby(['activity']).size())

ensemble
BO    1827
KP      46
KT       8
RE       2
SA      14
SP       4
dtype: int64


In [485]:
activity_list = ['knife_cutting', 'hammer_hammering', 'pen_writing', 'mug_drinking', 'spoon_stirring',
            'saw_sawing', 'screwdriver_screwing(release)', 'knife_chopping', 'bottle_drinking', 'none']

pose_list = ['knife_cutting', 'hammer_hammering', 'pen_writing', 'mug_drinking', 'spoon_stirring',
            'saw_sawing', 'screwdriver_screwing(no-release)', 'knife_chopping', 'bottle_drinking', 'none']

In [486]:
for i in range(10):
    print(activity_list[i])
    print(pose_list[i])
    print("\n")

knife_cutting
knife_cutting


hammer_hammering
hammer_hammering


pen_writing
pen_writing


mug_drinking
mug_drinking


spoon_stirring
spoon_stirring


saw_sawing
saw_sawing


screwdriver_screwing(release)
screwdriver_screwing(no-release)


knife_chopping
knife_chopping


bottle_drinking
bottle_drinking


none
none


